In [1]:
import re
import sqlite3
import string
import PyPDF2
import pandas as pd

### Loading The PDF files

In [2]:
filepath = "/home/charismatic/Downloads/Voter_Statistics_2022/rov_per_polling_station.pdf"
pdf_object = open(file=filepath, mode='rb')

# Reading the data from the pdf files
pages = PyPDF2.PdfReader(pdf_object).pages

# Getting the Number of Pages
numberOfPages = len(pages)

# Iterating through each page getting the text
cleaned_records = list()
for page_no in range(numberOfPages):
    page_data = pages[page_no].extract_text()
    # page_data = pages[489].extract_text()

    if page_no+1 == numberOfPages:
        splitted_page_data = page_data.splitlines()[3:-2]
    else:
        splitted_page_data = page_data.splitlines()[3:-1]

    # Looping though the contents of a single page
    for record in splitted_page_data:
        # Removing the extra spaces at the end of each record
        stripped_record = record.strip()

        # Check if the record is the last one on the page so that we can remove the footer
        if stripped_record.endswith("ELECTION"):
            stripped_record = stripped_record.split('REGISTERED')[0].strip()
        
        record_fields = stripped_record.split(' ')
        
        pattern = re.compile(r'\b\d{3,}[A-z]*\b')
        pattern2 = re.compile(r'\b\d{3,}')
        temp_codes_store = pattern.findall(stripped_record)
        codes_store = [(pattern2.findall(x)[0] if not x.isdigit() else x) for x in temp_codes_store]

        if len(codes_store) < 6:
            codes_store.append(record_fields[-1])

        if len(codes_store) > 6:
            codes_stores_j = [re.findall(r'^\b\d{3,}[A-z]*\b', x) for x in record_fields]
            codes_stores_k = [re.findall(r'\d{3,}',o[0])[0] for o in codes_stores_j if len(o) > 0]
            if len(codes_stores_k) > 6:
                codes_stores_l = [re.findall(r'\d{3,}', o)[0] for o in re.findall(r'\d{3,}\s*[A-z]{2,}', stripped_record)] 
                codes_stores_l.append(record_fields[-1])
                codes_store = codes_stores_l
            else:
                codes_store = codes_stores_k
            
        names_pattern = re.compile(r'(?<=\d)\s*([A-Za-z\s\/\?\.\-\(\)\'\&\"]+)\s*(?=\d)')
        names_store = [x.strip() for x in re.split(r'\b\d{2,}', stripped_record) if x != '' or x != ' ']
        if len(names_store) > 5:
            names_store = [x.strip() for x in re.split(r'\b\d{3,}', stripped_record) if x != '' or x != ' ']
        names_store = [x for x in names_store if x != '']
        if names_store[-1] != names_store[-2]:
            names_store[-1] = names_store[-2]
        try:
            cleaned_record = {
                'County_Code': codes_store[0], 'County_Name': names_store[0], 'Constituency_Code': codes_store[1], 'Constituency_Name': names_store[1], 
                'Ward_Code': codes_store[2], 'Ward_Name': names_store[2], 'Reg_Center_Code': codes_store[3], 'Reg_Center_Name': names_store[3], 
                'Station_Code': codes_store[4], 'Station_Name': names_store[4], 'Registered_Voters': codes_store[5]
            }
            
            cleaned_records.append(cleaned_record)
        except Exception as e:
            print("\nError Message")
            print(stripped_record)
            print(codes_store)
            print(names_store)
            raise e
# Convert the collected data into a dataframe
registered_voters_per_station_df = pd.DataFrame(cleaned_records)
registered_voters_per_station_df

,County_Code,County_Name,Constituency_Code,Constituency_Name,Ward_Code,Ward_Name,Reg_Center_Code,Reg_Center_Name,Station_Code,Station_Name,Registered_Voters
0,001,MOMBASA,001,CHANGAMWE,0001,PORT REITZ,001,BOMU PRIMARY SCHOOL,001001000100101,BOMU PRIMARY SCHOOL,673
1,001,MOMBASA,001,CHANGAMWE,0001,PORT REITZ,001,BOMU PRIMARY SCHOOL,001001000100102,BOMU PRIMARY SCHOOL,673
2,001,MOMBASA,001,CHANGAMWE,0001,PORT REITZ,001,BOMU PRIMARY SCHOOL,001001000100103,BOMU PRIMARY SCHOOL,673
3,001,MOMBASA,001,CHANGAMWE,0001,PORT REITZ,001,BOMU PRIMARY SCHOOL,001001000100104,BOMU PRIMARY SCHOOL,672
4,001,MOMBASA,001,CHANGAMWE,0001,PORT REITZ,001,BOMU PRIMARY SCHOOL,001001000100105,BOMU PRIMARY SCHOOL,672
...,...,...,...,...,...,...,...,...,...,...,...
46228,049,PRISONS,292,PRISONS,1451,PRISONS,113,KAMITI MAIN PRISON,049292145111302,KAMITI MAIN PRISON,362
46229,049,PRISONS,292,PRISONS,1451,PRISONS,116,NAIROBI REMAND/ALLOCATION,049292145111601,NAIROBI REMAND/ALLOCATION,81
46230,049,PRISONS,292,PRISONS,1451,PRISONS,118,KILGORIS PRISON,049292145111801,KILGORIS PRISON,12
46231,049,PRISONS,292,PRISONS,1451,PRISONS,119,LOITOKITOK GK PRISON,049292145111901,LOITOKITOK GK PRISON,3


In [3]:
registered_voters_per_station_df.to_csv("registered_voters.csv", index=False)

In [4]:
registered_voters_per_station_df.sort_values(by='Registered_Voters', ascending=True).head().reset_index(drop=True)

,County_Code,County_Name,Constituency_Code,Constituency_Name,Ward_Code,Ward_Name,Reg_Center_Code,Reg_Center_Name,Station_Code,Station_Name,Registered_Voters
0,049,PRISONS,292,PRISONS,1451,PRISONS,035,KITUI WOMEN,049292145103501,KITUI WOMEN,1
1,008,WAJIR,037,ELDAS,0182,LAKOLEY SOUTH/BASIR,049,OROTE PRIMARY SCHOOL,008037018204901,OROTE PRIMARY SCHOOL,1
2,049,PRISONS,292,PRISONS,1451,PRISONS,093,BUSIA WOMEN WING,049292145109301,BUSIA WOMEN WING,1
3,039,BUNGOMA,223,KIMILILI,1111,KIBINGEI,067,MUNGU SCHOOL,039223111106701,MUNGU SCHOOL,1
4,047,NAIROBI CITY,281,RUARAKA,1404,LUCKY SUMMER,024,BEGA KWA BEGA GLUCOLA,047281140402401,BEGA KWA BEGA GLUCOLA,1


### Inserting the data into a database

In [5]:
# Initiate a connection to the database
conn = sqlite3.connect('iebcrecords.db')
cursor = conn.cursor()

# Create the required table
cursor.execute(
    """
        CREATE TABLE IF NOT EXISTS Registered_Voters_Per_Station (
            County_Code VARCHAR(5) NOT NULL, 
            County_Name TEXT NOT NULL, 
            Constituency_Code VARCHAR(5) NOT NULL, 
            Constituency_name TEXT NOT NULL, 
            Ward_Code VARCHAR(5) NOT NULL, 
            Ward_Name TEXT NOT NULL, 
            Reg_Center_Code VARCHAR(5) NOT NULL, 
            Reg_Center_Name TEXT NOT NULL, 
            Station_Code VARCHAR(20) NOT NULL PRIMARY KEY, 
            Station_Name TEXT NOT NULL, 
            Registered_Voters INT
        )
    """
)
# Write the changes
conn.commit()

# Insert the data we will use a custom insert statement so as to ensure data integrity
voter_records = registered_voters_per_station_df.values
print("[+] Inserting voter records per polling station...")
for index, voter_record in enumerate(voter_records):
    try:
        cursor.execute(
            """
                INSERT OR IGNORE INTO Registered_Voters_Per_Station VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, voter_record
        )
        print(f"[+] Inserted {index+1}/{len(voter_records)}\r", end="")
    except Exception as e:
        conn.rollback()
        conn.close()
        raise e
conn.commit()
print("\n[+] Operation completed successfully")

[+] Inserting voter records per polling station...
[+] Inserted 46233/46233
[+] Operation completed successfully


### Creating views

In [6]:
# Registered Voters per County View
cursor.execute(
    """
        CREATE VIEW IF NOT EXISTS REGISTERED_VOTERS_PER_COUNTY AS 
        SELECT 
        	county_code, county_name, sum(Registered_Voters) 
        AS 
        	Voters_Registered 
        FROM 
        	Registered_Voters_Per_Station 
        GROUP BY 
        	county_code 
        UNION ALL
        	SELECT 
        		NULL AS county_code, 
        		'TOTAL_VOTERS' AS county_name, 
        		SUM(Registered_Voters) AS Voters_Registered
        	FROM 
        		Registered_Voters_Per_Station;
    """
)

# Create a view of registered voters per constituency
cursor.execute(
    """
        CREATE VIEW IF NOT EXISTS REGISTERED_VOTERS_PER_CONSTITUENCY AS 
        SELECT 
        	constituency_code, constituency_name, sum(Registered_Voters) 
        AS 
        	Voters_Registered 
        FROM 
        	Registered_Voters_Per_Station 
        GROUP BY 
        	constituency_code 
        UNION ALL
        	SELECT 
        		NULL AS constituency_code, 
        		'TOTAL_VOTERS' AS constituency_name, 
        		SUM(Registered_Voters) AS Voters_Registered
        	FROM 
        		Registered_Voters_Per_Station;
    """
)

# Create a view of registered voters per ward
cursor.execute(
    """
        CREATE VIEW IF NOT EXISTS REGISTERED_VOTERS_PER_WARD AS 
        SELECT 
        	ward_code, ward_name, sum(Registered_Voters) 
        AS 
        	Voters_Registered 
        FROM 
        	Registered_Voters_Per_Station 
        GROUP BY 
        	ward_code 
        UNION ALL
        	SELECT 
        		NULL AS ward_code, 
        		'TOTAL_VOTERS' AS ward_name, 
        		SUM(Registered_Voters) AS Voters_Registered
        	FROM 
        		Registered_Voters_Per_Station;
    """
)

# Create a view of registered voters per Registration center
cursor.execute(
    """
        CREATE VIEW IF NOT EXISTS REGISTERED_VOTERS_PER_REG_CENTER AS 
        SELECT 
        	Reg_Center_Code, Reg_Center_Name, sum(Registered_Voters) 
        AS 
        	Voters_Registered 
        FROM 
        	Registered_Voters_Per_Station 
        GROUP BY 
        	Reg_Center_Code 
        UNION ALL
        	SELECT 
        		NULL AS Reg_Center_Code, 
        		'TOTAL_VOTERS' AS Reg_Center_Name, 
        		SUM(Registered_Voters) AS Voters_Registered
        	FROM 
        		Registered_Voters_Per_Station;
    """
)
conn.commit()

### Close connection

In [7]:
conn.close()